## *Landsat* 系列数据去云
AIE平台中提供的 *Landsat* 影像均为 *USGS Collection 2* 中的数据，该集合使用 *QA_PIXEL* 波段对云、雪进行描述。因此，可通过位运算和逻辑运算，生成云掩膜，利用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=2ea198" target="_blank">updateMask</a>* 函数去除可能是云或者云阴影的区域。此算法适用于 *Landsat-5 / 7 / 8 / 9* 。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### 定义去云算法

使用 *QA_PIXEL* 波段进行去云处理。分别缩放SR波段和ST波段像元值，使SR波段像元值尽量在[0,1]之间，ST波段为地表温度的开尔文表示。

In [2]:
def removeLandsatCloud(image):
    cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudShadowBitMask)).eq(aie.Image(0)).And(qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0)))
    return image.updateMask(mask)

In [3]:
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(aie.Image(0.0000275)).add(aie.Image(-0.2))
    thermalBands = image.select('ST_B.*').multiply(aie.Image(0.00341802)).add(aie.Image(149.0))
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

### *Landsat* 数据检索
指定区域、时间、云量等，获取目标影像集。检索区域可通过 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=d0363b" target="_blank">aie.Geometry</a>* 构造、引用平台内置的行政边界文件或用户自主上传的矢量文件等。

In [4]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省')) 

In [5]:
dataset = aie.ImageCollection('LANDSAT_LC08_C02_T1_L2') \
             .filterBounds(region) \
             .filterDate('2018-01-01', '2018-10-31') \
             .filter(aie.Filter.lte('eo:cloud_cover', 20.0))

print(dataset.size().getInfo())

### 算法调用
对检索到的 *dataset* 进行去云算法到用并进行数据地图可视化显示。使用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=bc42ba#heading-0" target="_blank">aie.ImageCollection.map</a>* 函数可实现对影像集合中的每景影像循环执行指定函数，此处实现对全部检索到的 *Landsat8* 进行去云处理。



In [6]:
images_no_cloud = dataset.map(removeLandsatCloud).map(applyScaleFactors)
image = images_no_cloud.mosaic()

vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

map = aie.Map(
    center=image.getCenter(),
    height=800,
    zoom=5
)

map.addLayer(
    image,
    vis_params,
    'True Color(432)',
    bounds=image.getBounds()
)

map

Map(center=[29.582650056159693, 120.43692573504126], controls=(ZoomControl(options=['position', 'zoom_in_text'…